In [9]:
import json
from datetime import datetime, timedelta

import plotly.graph_objects as go

import lib.models as tm
import plotting
from lib.orchestra import load_orchestra

In [21]:
import importlib
importlib.reload(tm)
importlib.reload(plotting)

<module 'plotting' from 'c:\\Users\\kbarnard\\git\\lcm-parser\\plotting.py'>

In [2]:
print('Parsing to dicts...', end='', flush=True)
with open('may-24-25-tracks.json') as f:
    tracks = json.load(f)
print('done')

Parsing to dicts...done


In [11]:
orchestra = load_orchestra('orchestra_extended.pkl')

ValueError: Element state_num does not exist.

In [13]:
supervisor_state_signal = orchestra.get_signal('supervisor').get_signal('state_number')

In [3]:
# N = 1000
# start_dt = datetime.fromisoformat('2021-05-24T16:47:24+00:00')
# end_dt = datetime.fromisoformat('2021-05-24T17:05:21+00:00')
start_id = 608
end_id = 1184

# Filter tracks in range
filtered_tracks = filter(lambda t: start_id <= int(t['id']) <= end_id, tracks)

In [4]:

print('Parsing to Track objects...', end='', flush=True)
filtered_tracks = [tm.Track.from_dict(track) for track in filtered_tracks]
print('done')

Parsing to Track objects...done


In [5]:
len(filtered_tracks)

391

In [6]:
# Sort tracks by closeness to 15 seconds in duration
target_duration = timedelta(seconds=15)

sorted_tracks = sorted(filtered_tracks, key=lambda t: abs(t.duration - target_duration))

In [7]:
# Select only tracks with > 10 unique points
sorted_tracks = [t for t in sorted_tracks if len(t.unique_points) > 10]

len(sorted_tracks)

127

In [17]:
# Select target IDs
target_ids = [t.id for t in sorted_tracks]

# Get signals in orchestra
tracks_orchestra = orchestra.get_signal('offset_tracks')
track_orchestras = [tracks_orchestra.get_signal(str(id)) for id in target_ids]

In [26]:
traces = [
    trace 
    for track_id, track_orchestra in zip(target_ids[:50], track_orchestras) 
    for trace in plotting.track_orchestra_to_trace_list(track_id, track_orchestra, supervisor_state_signal)
]

In [27]:
fig = go.Figure(data=traces)
fig.update_yaxes(scaleanchor="x", scaleratio=1)
fig.update_layout(height=800, width=800)
fig.show()